In [1]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="quick-starts-ws-148187")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="Project1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-150299
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-150299


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "ML-Cluster"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, loguniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": loguniform(0.1, 10),
    "--max_iter": choice(50,100,200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn("./", compute_target=cpu_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                    estimator=est, 
                                    policy=policy,
                                    primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  
                                    max_total_runs=32)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [7]:
import joblib
# Get your best run and save the model from that run.

best_run = hdr.get_best_run_by_primary_metric()
best_run_metric = best_run.get_metrics()

print(best_run)
print(best_run_metric)
best_run.get_file_names()
best_model=best_run.register_model(model_name="best_hdr_model", model_path=best_run.get_file_names()[-1], tags=best_run_metric)

# Save the best model
best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/')

Run(Experiment: Project1,
Id: HD_2a1dc824-f694-4973-80a8-928f2e01f5f1_27,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 5448.578094934016, 'Max iterations:': 50, 'Accuracy': 0.9041029376062151}


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

datastore_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path)

In [9]:
from train import clean_data
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y)

AMLdata = x_train.join(y_train)
AMLdata.head()

,age,marital,default,housing,loan,month,day_of_week,campaign,pdays,previous,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
31722,42,0,0,1,0,11,5,1,999,1,...,0,0,0,0,0,0,1,0,0,0
2995,37,1,0,1,0,11,1,1,999,0,...,0,0,0,0,0,0,0,1,0,0
18100,55,1,0,0,0,7,4,1,999,0,...,0,1,0,0,0,0,0,0,0,0
6342,55,1,0,1,0,7,2,3,999,0,...,0,0,0,0,0,1,0,0,0,0
2675,45,1,0,0,0,11,5,1,999,0,...,0,0,0,1,0,0,0,0,0,0


In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=AMLdata,
    label_column_name='y',
    n_cross_validations=3)

In [11]:
# Submit your automl run

ws = Workspace.from_config()
experiment = Experiment(ws, "Project1-AML")
run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
Project1-AML,AutoML_6d7ea279-15c5-402a-b272-298989c518c8,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [12]:
# Retrieve and save your best automl model.

best_run, best_model = run.get_output()
print(best_run.get_metrics(name="accuracy"))
print(run.summary())
print(best_model.steps[-1])

{'accuracy': 0.9021124451143688}
[['StackEnsemble', 1, 0.9016268215953946], ['VotingEnsemble', 1, 0.9021124451143688], ['XGBoostClassifier', 22, 0.9012221582538084], ['LightGBM', 16, 0.90101981430218], ['ExtremeRandomTrees', 3, 0.9002104925313413], ['RandomForest', 7, 0.8997249132233734], ['LogisticRegression', 1, 0.8994011953221729], ['GradientBoosting', 1, 0.891550740249933]]
('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(
    estimators=[('20', Pipeline(memory=None,
             steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')),
                    ('xgboostclassifier',
                     XGBoostClassifier(booster='gbtree', colsample_bytree=0.6, eta=0.4, gamma=0, max_depth=6, max_leaves=0, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(
        dataset_samples=24712,
        dataset_features=119,
        dataset_classes=2,
        dataset_num_cat...
        iteration_timeout_mode=0,
        iteration_timeout_param=None,
 

In [14]:
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

